In [1]:
import csv
from scipy import ndimage
from scipy import linalg
import numpy as np
import nibabel as nib
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter
import pickle

In [2]:
import random

index = np.array(range(817))
random.seed(40)
random.shuffle(index)

train_index = index[0:572]
val_index = index[572:654]
test_index = index[654:817]

In [3]:
#dataimage_train = [dataimage[j] for j in train_index]
#print(dataimage_train)

In [4]:
lab = np.zeros((817,1)) #first AD:188 then CN:288
lab[0:188, 0] = 1 #AD -- 1; CN -- 0; MCI -- 2
lab[416:817, 0] = 2
for i in range(0, 817):
    print(i, lab[i])

0 [1.]
1 [1.]
2 [1.]
3 [1.]
4 [1.]
5 [1.]
6 [1.]
7 [1.]
8 [1.]
9 [1.]
10 [1.]
11 [1.]
12 [1.]
13 [1.]
14 [1.]
15 [1.]
16 [1.]
17 [1.]
18 [1.]
19 [1.]
20 [1.]
21 [1.]
22 [1.]
23 [1.]
24 [1.]
25 [1.]
26 [1.]
27 [1.]
28 [1.]
29 [1.]
30 [1.]
31 [1.]
32 [1.]
33 [1.]
34 [1.]
35 [1.]
36 [1.]
37 [1.]
38 [1.]
39 [1.]
40 [1.]
41 [1.]
42 [1.]
43 [1.]
44 [1.]
45 [1.]
46 [1.]
47 [1.]
48 [1.]
49 [1.]
50 [1.]
51 [1.]
52 [1.]
53 [1.]
54 [1.]
55 [1.]
56 [1.]
57 [1.]
58 [1.]
59 [1.]
60 [1.]
61 [1.]
62 [1.]
63 [1.]
64 [1.]
65 [1.]
66 [1.]
67 [1.]
68 [1.]
69 [1.]
70 [1.]
71 [1.]
72 [1.]
73 [1.]
74 [1.]
75 [1.]
76 [1.]
77 [1.]
78 [1.]
79 [1.]
80 [1.]
81 [1.]
82 [1.]
83 [1.]
84 [1.]
85 [1.]
86 [1.]
87 [1.]
88 [1.]
89 [1.]
90 [1.]
91 [1.]
92 [1.]
93 [1.]
94 [1.]
95 [1.]
96 [1.]
97 [1.]
98 [1.]
99 [1.]
100 [1.]
101 [1.]
102 [1.]
103 [1.]
104 [1.]
105 [1.]
106 [1.]
107 [1.]
108 [1.]
109 [1.]
110 [1.]
111 [1.]
112 [1.]
113 [1.]
114 [1.]
115 [1.]
116 [1.]
117 [1.]
118 [1.]
119 [1.]
120 [1.]
121 [1.]
122 [1.]
123

In [5]:
mci = 0.
ad = 0.
cn = 0.
total_num = 0.
for i in lab:
    if(i == 1):
        ad += 1
    elif(i == 2):
        mci += 1
    elif(i == 0):
        cn += 1
total_num = mci + ad + cn
print(mci, ad, cn, total_num)

401.0 188.0 228.0 817.0


In [6]:
from torch.utils.data import Dataset, DataLoader

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

class ADNI_Dataset(Dataset):
    
    def __init__(self, ptype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #if(ptype == 'train'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        
        #if(ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
        self.ptype = ptype
        
        self.img = np.load('GRAY_AD_CN_MCI_Jan27.npy') # 416 * ((121 * 121) * 3)
        self.lab = np.zeros((817,1)) #first AD:188 then CN:288
        self.lab[0:188, 0] = 1 #AD -- 1; CN -- 0; MCI -- 2
        self.lab[416:817, 0] = 2
        #for i in range(188):
            #lab[i] = 1
        if(self.ptype == 'train'):
            self.img = self.img[train_index]
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            for i in range(572):
                for j in range(3):
                    self.img[i][j] = self.img[i][j] + np.random.normal(0, 0.01, (121, 121))
                    print(self.img[i][j].shape)
                    #count += 1
            #print(count)
            self.lab = self.lab[train_index]
        if(self.ptype == 'val'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[val_index]
            self.lab = self.lab[val_index]
        if(self.ptype == 'test'):
            #img = np.load('GRAY_AD_CN_Jan25.npy')
            self.img = self.img[test_index]
            self.lab = self.lab[test_index]
        
        print("great job dude")
        

    def __len__(self):#return int: number of mri
        return len(self.img)

    def __getitem__(self, idx):#input the dataset and desires i-th item
       
        sample = {'image': self.img[idx], 'labels': self.lab[idx,0]} 

        return sample


In [7]:
ADNI_Data_train = ADNI_Dataset('train')

ADNI_Data_val = ADNI_Dataset('val')

ADNI_Data_test = ADNI_Dataset('test')



dataloader_train = DataLoader(ADNI_Data_train, batch_size=400,
                        shuffle=True, num_workers=4)
dataloader_val = DataLoader(ADNI_Data_val, batch_size=400,
                        shuffle=True, num_workers=4)
dataloader_test = DataLoader(ADNI_Data_test, batch_size=400,
                        shuffle=True, num_workers=4)

print("finished")

(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)

(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)

(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)
(121, 121)

In [8]:
import torch.nn as nn
import torch.nn.functional as F

#initial: 
#121 * 121 * 3(feature map number)
#60 * 60 * 16
#30 * 30 * 32
#15 * 15 * 64
#7 * 7 * 128
#3 * 3 * 256

#Linear: 3 * 3 * 256
#3 * 3 * 256(2304)--->1024 --->256 ---> 64 --->3 
class Net_2D(nn.Module):
    def __init__(self):
        super(Net_2D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 5, padding=2)
        self.conv2 = nn.Conv2d(16, 32, 5, padding=2)
        self.conv3 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv4 = nn.Conv2d(64, 128, 5, padding=2)
        self.conv5 = nn.Conv2d(128, 256, 5, padding=2)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(3 * 3 * 256, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 3)

    def forward(self, x): #batch_size * 256*256*160*1
        x = self.pool(F.relu(self.conv1(x)))#batch_size * 128 * 128 * 80 * 2
        x = self.pool(F.relu(self.conv2(x)))#batch_size * 64 * 64 * 40 * 4
        x = self.pool(F.relu(self.conv3(x)))#batch_size * 32 * 32 * 20 * 8
        x = self.pool(F.relu(self.conv4(x)))#batch_size * 16 * 16 * 10 * 16
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(-1, 3 * 3 * 256)#batch_size * 10240(8*8*5*32)
        x = F.relu(self.fc1(x))#batch_size * 1024
        x = F.relu(self.fc2(x))#batch_size * 256
        x = F.relu(self.fc3(x))#batch_size * 64
        x = self.fc4(x)#batch_size * 3
        return x

net_2d = Net_2D().cuda()

print("finished")

finished


In [9]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import math


__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


def alexnet(pretrained=True, **kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = AlexNet(**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['alexnet']))
        for p in model.features.parameters():
            p.requires_grad = False

        # fine-tune the last convolution layer
        for p in model.features[10].parameters():
            p.requires_grad = True

    model.classifier.add_module('fc_out', nn.Linear(1000,3))
    model.classifier.add_module('sigmoid', nn.LogSoftmax())

    stdv = 1.0 / math.sqrt(1000)
    for p in model.classifier.fc_out.parameters():
        p.data.uniform_(-stdv, stdv)

    return model

In [10]:
import torch.optim as optim
import torch 
Weight = torch.tensor([total_num / cn, total_num / ad, total_num / mci])
criterion = nn.CrossEntropyLoss(weight = Weight.cuda())
optimizer = optim.SGD(net_2d.parameters(), lr=0.001, momentum=0.9)

print("finished")

finished


In [11]:
import numpy as np
import torch

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

loss_val_min = 1000000
threshold = 100

epoch = 50000
min_total_loss = 1000000


for i in range(epoch):
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched[images].shape)
        #print(i_batch)
        #print(i_batch,
        #      sample_batched['patient_id'])
        
        labels = sample_batched['labels'].cuda()
        #labels_arr = np.asarray(sample_batched['labels'])
        #print(labels_arr.shape)
        running_loss = 0.0
       
    # get the inputs
        
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        images = sample_batched['image'].float().cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()



        # zero the parameter gradients
        optimizer.zero_grad()
        
        #before actual training
        net_2d.train()

        # forward + backward + optimize
        outputs = net_2d(images)
        #print(outputs)
        #print(labels.long())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))

    print('$$Finished Training This Epoch')
    
    #validation for the epoch
    net_2d.eval()
    total_val_loss = 0
    print('Start a new val_epoch')
    for i_batch, sample_batched in enumerate(dataloader_val):
        #print(i_batch)
              #sample_batched['patient_id'])

        running_loss = 0.0
        
        labels = sample_batched['labels'].cuda()
        images = sample_batched['image'].float().cuda()
        # get the inputs
        #labels_arr = np.asarray(sample_batched['labels'])
        #labels_arr = labels_arr.astype(np.int)
        #labels = ToTensor(labels_arr).cuda()

        #images_arr = np.asarray(sample_batched['image'])
        #images_arr = images_arr.astype(np.float)
        #images_arr = np.expand_dims(images_arr, 1)
        #images = ToTensor(images_arr).cuda()
        
        outputs = net_2d(images)
        loss = criterion(outputs, labels.long())
        print('Validation')
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))
        #print(loss.item())
        #print(type(loss.item()))
        print('\n')
        
        total_val_loss = total_val_loss + loss.cpu().data.numpy()
        
    if(total_val_loss >= loss_val_min):
        if(total_val_loss < 0.5):
            count += 1
    else:
        loss_val_min = total_val_loss
        count = 0
        torch.save(net_2d, 'best_model_Feb2_NOISE_3Type_weighted_nonoverfit.pkl')
    #if(total_val_loss < min_total_loss):
        #min_total_loss = total_val_loss
        #save best model
        #torch.save(net_2d, 'best_model_Jan27_2d_3types.pkl')
    if(count > threshold):
        break
    

print("we're done training")

[1] loss: 1.100
[2] loss: 1.097
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.097




/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net_2D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[1] loss: 1.098
[2] loss: 1.102
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.097


[1] loss: 1.099
[2] loss: 1.100
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.097


[1] loss: 1.100
[2] loss: 1.097
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.097


[1] loss: 1.099
[2] loss: 1.101
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.097


[1] loss: 1.099
[2] loss: 1.100
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.097


[1] loss: 1.100
[2] loss: 1.098
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.097


[1] loss: 1.098
[2] loss: 1.102
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.097


[1] loss: 1.099
[2] loss: 1.101
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.097


[1] loss: 1.099
[2] loss: 1.100
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 1.099
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.098
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.098
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.098
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 1.098
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.098
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.098
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.099
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.098
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.098
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.098
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 1.098


[1] loss: 1.099
[2] loss: 1.098
$$Finished Training This Epoch
Start a new val_epoch
Val

...

[1] loss: 0.422
[2] loss: 0.437
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.406


[1] loss: 0.416
[2] loss: 0.446
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.409


[1] loss: 0.422
[2] loss: 0.428
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.409


[1] loss: 0.415
[2] loss: 0.443
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.406


[1] loss: 0.418
[2] loss: 0.428
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.406


[1] loss: 0.418
[2] loss: 0.433
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.418


[1] loss: 0.427
[2] loss: 0.425
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.410


[1] loss: 0.410
[2] loss: 0.448
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.407


[1] loss: 0.401
[2] loss: 0.485
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.413
[2] loss: 0.450
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.405


[1] loss: 0.391
[2] loss: 0.494
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.404


[1] loss: 0.409
[2] loss: 0.439
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.415


[1] loss: 0.445
[2] loss: 0.376
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.404


[1] loss: 0.431
[2] loss: 0.381
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.404


[1] loss: 0.423
[2] loss: 0.398
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.404


[1] loss: 0.427
[2] loss: 0.391
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.405


[1] loss: 0.418
[2] loss: 0.408
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.404


[1] loss: 0.402
[2] loss: 0.448
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.383
[2] loss: 0.474
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.413


[1] loss: 0.416
[2] loss: 0.421
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.403


[1] loss: 0.406
[2] loss: 0.418
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.404


[1] loss: 0.417
[2] loss: 0.391
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.404


[1] loss: 0.423
[2] loss: 0.379
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.403


[1] loss: 0.391
[2] loss: 0.470
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.409


[1] loss: 0.402
[2] loss: 0.445
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.407


[1] loss: 0.407
[2] loss: 0.460
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.402


[1] loss: 0.404
[2] loss: 0.424
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.411
[2] loss: 0.391
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.403


[1] loss: 0.431
[2] loss: 0.339
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.402


[1] loss: 0.390
[2] loss: 0.433
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.405


[1] loss: 0.390
[2] loss: 0.445
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.402


[1] loss: 0.414
[2] loss: 0.388
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.402


[1] loss: 0.401
[2] loss: 0.432
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.402


[1] loss: 0.427
[2] loss: 0.374
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.401


[1] loss: 0.406
[2] loss: 0.406
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.412


[1] loss: 0.410
[2] loss: 0.394
$$Finished Training This Epoch
Start a new val_epoch
Val

[1] loss: 0.431
[2] loss: 0.389
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.403


[1] loss: 0.400
[2] loss: 0.399
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.410


[1] loss: 0.395
[2] loss: 0.447
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.404


[1] loss: 0.415
[2] loss: 0.374
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.405


[1] loss: 0.417
[2] loss: 0.363
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.405


[1] loss: 0.409
[2] loss: 0.400
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.402


[1] loss: 0.409
[2] loss: 0.374
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.403


[1] loss: 0.403
[2] loss: 0.387
$$Finished Training This Epoch
Start a new val_epoch
Validation
[1] loss: 0.404


[1] loss: 0.390
[2] loss: 0.420
$$Finished Training This Epoch
Start a new val_epoch
Val

In [12]:
import torch
#test after finish all epochs

#load the best_model back 
final_net = torch.load('best_model_Feb2_NOISE_3Type_weighted_nonoverfit.pkl')
final_net.eval()
correct_sum = 0.
total_sum = 0.
for i_batch, sample_batched in enumerate(dataloader_test):
    #print(i_batch)

    running_loss = 0.0
    labels = sample_batched['labels'].cuda()
    # get the inputs
    # labels_arr = np.asarray(sample_batched['labels'])
    # labels_arr = labels_arr.astype(np.int)
    # labels = ToTensor(labels_arr).cuda()
    images = sample_batched['image'].float().cuda()
    # images_arr = np.asarray(sample_batched['image'])
    # images_arr = images_arr.astype(np.float)
    # images = ToTensor(images_arr).cuda()

    outputs = final_net(images)#n * 3
    results_type = np.argmax(outputs.cpu().data.numpy(), axis=1)
    results_type = ToTensor(results_type)
    #print(outputs)

    #calculate accuracy
    labels = labels.cpu()
    print("predicted:\n", results_type, "\nactual:\n", labels)

correct_sum_AD = 0
correct_sum_CN = 0
correct_sum_MCI = 0

AD_num = 0
MCI_num = 0
CN_num = 0
for label in labels:
    if (label.float() == 1):
        AD_num += 1
    if (label.float() == 0):
        CN_num += 1
    if (label.float() == 2):
        MCI_num += 1
print("labels", AD_num, CN_num, MCI_num)

AD_num = 0
MCI_num = 0
CN_num = 0
correct_sum_AD_CN = 0
correct_sum_AD_MCI = 0
correct_sum_CN_AD = 0
correct_sum_CN_MCI = 0
correct_sum_MCI_CN = 0
correct_sum_MCI_AD = 0
for results in results_type:
    if (results.float() == 1):
        AD_num += 1
    if (results.float() == 0):
        CN_num += 1
    if (results.float() == 2):
        MCI_num += 1
#print("predicted", AD_num, CN_num, MCI_num)
for i in range(len(results_type)):
    if(labels[i].float() == results_type[i].float()):
        correct_sum += 1

for i in range(len(results_type)):
    if(labels[i].float() == 1):
        if(results_type[i].float() == 1):
            correct_sum_AD += 1
        if(results_type[i].float() == 0):
            correct_sum_AD_CN += 1
        if(results_type[i].float() == 2):
            correct_sum_AD_MCI += 1
    if(labels[i].float() == 0):
        if(results_type[i].float() == 0):
            correct_sum_CN += 1
        if(results_type[i].float() == 1):
            correct_sum_CN_AD += 1
        if(results_type[i].float() == 2):
            correct_sum_CN_MCI += 1
    if(labels[i].float() == 2):
        if(results_type[i].float() == 2):
            correct_sum_MCI += 1
        if(results_type[i].float() == 1):
            correct_sum_MCI_AD += 1
        if(results_type[i].float() == 0):
            correct_sum_MCI_CN += 1
    total_sum += 1
    
print("correct_sum", correct_sum)
print("correct_sum_AD", correct_sum_AD, correct_sum_AD_CN, correct_sum_AD_MCI)
print("correct_sum_CN", correct_sum_CN_AD, correct_sum_CN, correct_sum_CN_MCI)
print("correct_sum_MCI", correct_sum_MCI_AD, correct_sum_MCI_CN, correct_sum_MCI)
print(total_sum)


predicted:
 tensor([0., 2., 2., 2., 0., 1., 0., 2., 2., 2., 1., 2., 1., 1., 2., 0., 0., 2.,
        1., 0., 0., 2., 2., 2., 2., 0., 2., 2., 2., 2., 1., 2., 2., 1., 1., 0.,
        1., 2., 0., 2., 2., 2., 0., 2., 2., 2., 0., 2., 2., 1., 0., 0., 2., 2.,
        1., 0., 1., 0., 2., 1., 0., 1., 1., 2., 2., 0., 0., 0., 2., 0., 2., 1.,
        2., 1., 2., 0., 0., 1., 0., 1., 2., 1., 2., 0., 2., 0., 0., 2., 2., 0.,
        0., 2., 0., 2., 0., 2., 0., 2., 2., 2., 2., 2., 0., 2., 1., 2., 0., 0.,
        0., 1., 2., 2., 1., 1., 2., 0., 0., 2., 1., 1., 1., 1., 2., 2., 0., 2.,
        1., 0., 2., 0., 2., 2., 2., 0., 0., 2., 1., 0., 1., 2., 1., 1., 0., 0.,
        0., 0., 2., 1., 2., 2., 2., 0., 2., 2., 2., 0., 0., 0., 2., 2., 2., 2.,
        1.]) 
actual:
 tensor([0., 2., 2., 0., 0., 1., 0., 2., 2., 2., 1., 0., 1., 1., 2., 0., 2., 2.,
        1., 0., 2., 0., 2., 2., 2., 0., 0., 2., 0., 0., 1., 2., 2., 1., 1., 2.,
        1., 2., 2., 2., 2., 0., 0., 2., 2., 0., 0., 2., 2., 1., 2., 0., 2., 0.,
     